In [1]:
import at
import sys
from at import Param, ParamArray
from importlib.resources import files, as_file
from timeit import timeit

In [2]:
fname = 'hmba.mat'
with as_file(files('machine_data') / fname) as path:
    ring = at.load_lattice(path)

# Parameters

Parameters are objects of class {py:class}`.Param` which can be used instead of numeric values as {py:class}`.Element` attributes.

Parameters are initialised with a **scalar** numeric value. They have an optional name, only used to identify them in printed output:

In [3]:
total_length = Param(2.5, name="total_length")
dlength = Param(1.0, name="dlength")
print(f"{total_length}: {total_length!r}")
print(f"{dlength}: {dlength!r}")

total_length: 2.5
dlength: 1.0


The value of a parameter can be read or modified through its {py:attr}`~.variables.Variable.value` property. {py:meth}`~.variables.Variable.set` and {py:meth}`~.variables.Variable.get` methods are also available:

In [4]:
print(total_length.value)
total_length.value = 2.4
print(f"{total_length}: {total_length!r}")
total_length.set(2.3)
print(total_length.get())

2.5
total_length: 2.4
2.3


Arithmetic combinations of parameters create new read-only parameters of class {py:class}`.ParamBase`, whose value is permanently kept up-to-date:

In [5]:
qlength = total_length - dlength
print(f"{qlength}: {qlength!r}")
dlength.value = 0.9
print(f"{qlength}: {qlength!r}")

total_length-dlength: 1.2999999999999998
total_length-dlength: 1.4


Parameters may be assigned to {py:class}`.Element` attributes, for instance on initialisation:

In [6]:
dr1 = at.Drift('DR1', dlength)
qf1 = at.Quadrupole('QF1', qlength, 0.6)
print(dr1)
print(qf1)

Drift:
       FamName: DR1
        Length: dlength
    PassMethod: DriftPass
Quadrupole:
       FamName: QF1
        Length: total_length-dlength
    PassMethod: StrMPoleSymplectic4Pass
      MaxOrder: 1
   NumIntSteps: 10
      PolynomA: [0. 0.]
      PolynomB: [0.  0.6]
             K: 0.6


The {py:class}`.Element` attributes keep their type so that all the processing of elements either in python functions or in C integrators is unchanged:

In [7]:
print(dr1.Length, type(dr1.Length))

0.9 <class 'float'>


## Assigning parameters

### To a single element
Parameters may be assigned to {py:class}`.Element` attributes in several ways:

**At element creation:**

In [8]:
dr2 = at.Drift('DR2', dlength)

**By converting a numeric attribute into a parameter:**

In [9]:
qd1 = at.Quadrupole('QD1', 0.5, -0.4)
ql = qd1.parameterise('Length')
print(ql)

param3


**By normal assignment, only for scalar parameters:**

In [10]:
qd1.Length = Param(0.2)
print(qd1)

Quadrupole:
       FamName: QD1
        Length: param4
    PassMethod: StrMPoleSymplectic4Pass
      MaxOrder: 1
   NumIntSteps: 10
      PolynomA: [0. 0.]
      PolynomB: [ 0.  -0.4]
             K: -0.4


**With the {py:meth}`~.Element.set_parameter` method:**

In [11]:
qstrength = Param(-0.5, name='quad_strength')
qd1.set_parameter('PolynomB', qstrength, index=1)
print(qd1)

Quadrupole:
       FamName: QD1
        Length: param4
    PassMethod: StrMPoleSymplectic4Pass
      MaxOrder: 1
   NumIntSteps: 10
      PolynomA: [0. 0.]
      PolynomB: [0.0 quad_strength]
             K: -0.5


### To selected elements of a {py:class}`.Lattice`
To act on several elements in a single step, {py:class}`.Lattice` methods similar to {py:class}`.Element` methods are available

**Convert numeric attributes into parameters:**

The attribute of all the selected elements is replaced by a single parameter whose initial value is the average of the original values.

In [12]:
kf1 = ring.parameterise('QF1[AE]', 'PolynomB', index=1, name='kf1')
print(f"{kf1}: {kf1!r}")
print(ring[5])

kf1: np.float64(2.5394599781303304)
Quadrupole:
       FamName: QF1A
        Length: 0.311896
    PassMethod: StrMPoleSymplectic4Pass
FringeQuadEntrance: 1
FringeQuadExit: 1
      MaxOrder: 1
   NumIntSteps: 20
      PolynomA: [0. 0.]
      PolynomB: [0.0 kf1]
             K: 2.5394599781303304


**Use the {py:meth}`~.Lattice.set_parameter` method:**

The attribute of all the selected elements is replaced by the provided parameter,

In [13]:
lf1 = Param(0.311896, name='lf1')
ring.set_parameter('QF1[AE]', 'Length', lf1)
print(ring[117])

Quadrupole:
       FamName: QF1E
        Length: lf1
    PassMethod: StrMPoleSymplectic4Pass
FringeQuadEntrance: 1
FringeQuadExit: 1
      MaxOrder: 1
   NumIntSteps: 20
      PolynomA: [0. 0.]
      PolynomB: [0.0 kf1]
             K: 2.5394599781303304


Once a Parameter is assigned to an attribute, it acquires the type and the constraints of
the attribute. For instance:

In [14]:
num_int_steps = Param(14.4, name="num_int_steps")
ring.set_parameter(at.Multipole, 'NumIntSteps', num_int_steps)
print(ring[5].NumIntSteps, type(ring[5].NumIntSteps))

14 <class 'int'>


In [15]:
num_int_steps.value = -5

ValueError: Value must be greater of equal to 0

The parameter behaves as the attribute.

## Retrieving parameters

Since the values of {py:class}`.Element` attributes keep their original type, they cannot be used to access the underlying parameter. The only way to retrieve it is to use the {py:meth}`~.Element.get_parameter` method. a {py:obj}`TypeError` is raised if the attribute is not
a Parameter.

In [16]:
ql = qf1.get_parameter('Length')
print(f"{ql}: {ql!r}")
print("ql is qlength:", ql is qlength)

total_length-dlength: 1.4
ql is qlength: True


This also works for items in array attributes:

In [17]:
qs = qd1.get_parameter('PolynomB', index=1)
print(f"{qs}: {qs!r}")
print("qs is qstrength:", qs is qstrength)

quad_strength: -0.5
qs is qstrength: True


## Checking parametrisation

The {py:meth}`~.Element.is_parameterised` method may be applied to:
- a full element: it returns true is any of its attributes is a parameter,
- an array attribute: it returns true if any of its items is a parameter,
- a scalar attribute or an item of an array.

In [18]:
print(qd1.is_parameterised())
print(qd1.is_parameterised('PolynomB', index=1))
print(qd1.is_parameterised('PolynomB', index=0))

True
True
False


## Removing parameters
Removing the parameters will "freeze" the element at its current value. The {py:meth}`~.Element.unparameterise` method is defined for both {py:class}`.Element` and {py:class}`.Lattice`, and may be applied to:
- a full element: all the parameters are replaced by their value,
- an array attribute: the whole {py:class}`.ParamArray` is replaced by a numpy array,
- a scalar attribute or an item of an array.

### In an Element

In [19]:
print(qd1)
qd1.unparameterise('Length')
print(qd1)
qd1.unparameterise('PolynomB', 1)
print(qd1)

Quadrupole:
       FamName: QD1
        Length: param4
    PassMethod: StrMPoleSymplectic4Pass
      MaxOrder: 1
   NumIntSteps: 10
      PolynomA: [0. 0.]
      PolynomB: [0.0 quad_strength]
             K: -0.5
Quadrupole:
       FamName: QD1
        Length: 0.2
    PassMethod: StrMPoleSymplectic4Pass
      MaxOrder: 1
   NumIntSteps: 10
      PolynomA: [0. 0.]
      PolynomB: [0.0 quad_strength]
             K: -0.5
Quadrupole:
       FamName: QD1
        Length: 0.2
    PassMethod: StrMPoleSymplectic4Pass
      MaxOrder: 1
   NumIntSteps: 10
      PolynomA: [0. 0.]
      PolynomB: [ 0.  -0.5]
             K: -0.5


### In a Lattice

In [20]:
print(ring[5])
ring.unparameterise('QF1[AE]', 'PolynomB', index=1)
print(ring[5])
ring.unparameterise('QF1[AE]')
print(ring[117])

Quadrupole:
       FamName: QF1A
        Length: lf1
    PassMethod: StrMPoleSymplectic4Pass
FringeQuadEntrance: 1
FringeQuadExit: 1
      MaxOrder: 1
   NumIntSteps: num_int_steps
      PolynomA: [0. 0.]
      PolynomB: [0.0 kf1]
             K: 2.5394599781303304
Quadrupole:
       FamName: QF1A
        Length: lf1
    PassMethod: StrMPoleSymplectic4Pass
FringeQuadEntrance: 1
FringeQuadExit: 1
      MaxOrder: 1
   NumIntSteps: num_int_steps
      PolynomA: [0. 0.]
      PolynomB: [0.         2.53945998]
             K: 2.5394599781303304
Quadrupole:
       FamName: QF1E
        Length: 0.311896
    PassMethod: StrMPoleSymplectic4Pass
FringeQuadEntrance: 1
FringeQuadExit: 1
      MaxOrder: 1
   NumIntSteps: 14
      PolynomA: [0. 0.]
      PolynomB: [0.         2.53945998]
             K: 2.5394599781303304


## Parameter history
Parameter values are kept in an {py:attr}`~.variables.Variable.history` buffer. The properties {py:attr}`~.variables.Variable.initial_value`, {py:attr}`~.variables.Variable.last_value` and {py:attr}`~.variables.Variable.previous_value` are also available:

In [21]:
dlength.value = 1.1
print(dlength.history)
print(dlength.initial_value)
print(dlength.previous_value)

[1.0, 1.0, 0.9, 1.1]
1.0
0.9


After varying parameters, in matching for instance, the current status can be printed:

In [22]:
print(dlength.status())


        Name      Initial          Final        Variation

     dlength    1.000000e+00    1.100000e+00    1.000000e-01


Parameters may be reset to a previous history value with the {py:meth}`~.variables.Variable.reset` and {py:meth}`~.variables.Variable.set_previous` methods. The history is shortened accordingly.

In [23]:
dlength.set_previous()
print(dlength, dlength.history)
dlength.reset()
print(dlength, dlength.history)

dlength [1.0, 1.0, 0.9]
dlength [1.0]


## Copying and Saving

Parameters cannot be copied. A shallow or deep copy of a parameter returns the parameter itself.

So in a deep copy of an {py:class}`.Element`, the parameters are preserved.

When saving a lattice with parameterised elements, as a `.mat`, a `.m` or `.repr` file, all parametrisation is removed, and a "frozen" state of the lattice is saved.

In pickle dumps of an {py:class}`.Element`, parameters are preserved.

In [24]:
a=timeit("b=qd1.NumIntSteps", globals=globals(), number=10000)
print(f"Scalar attribute, non-parameterised {a:.4}")
b=timeit("b=qd1.Length", globals=globals(), number=10000)
print(f"Scalar attribute,     parameterised {b:.4} {b/a:.3}")
a=timeit("b=qd1.PolynomA[1]", globals=globals(), number=10000)
print(f"Array attribute,  non-parameterised {a:.4}")
b=timeit("b=qd1.PolynomB[1]", globals=globals(), number=10000)
print(f"Array attribute,      parameterised {b:.4} {b/a:.3}")

Scalar attribute, non-parameterised 0.0003796
Scalar attribute,     parameterised 0.0004329 1.14
Array attribute,  non-parameterised 0.001071
Array attribute,      parameterised 0.001086 1.01
